In [ ]:
'''web scraping using beautiful soap'''

import requests
from bs4 import BeautifulSoup

def extract_text_from_url(url):
    response = requests.get(url, timeout=10)
    soup = BeautifulSoup(response.text, "html.parser")
    text = " ".join([p.get_text() for p in soup.find_all("p")])
    return text.lower()


url = "https://www.baunfire.com/"
text = extract_text_from_url(url)
f=open('markdown4.txt','w',encoding="utf-8")
f.write(text)
f.close()
automate=Automation('markdown4.txt')
response = automate.prediction()
print("########################################")
print(response)

print(text)  


In [ ]:
# prompt 1 (2/3)
"""Analyze the following website content and determine if it belongs to a Digital Marketing Agency.

        Look for similar keywords: Services, Web Design, Web Development, SEO Agency, Ads Agency, Digital, Marketing , Agency, Website Creation,
        brands,web,design,Boosts creativity,Reduces costs.
        
        If most of the keywords appear in the website content then classify it as *"Digital Marketing Agency"*.
        Otherwise classify it as *"Not a Digital Marketing Agency"* and format output as given below.
        No explanations.

       format the output as a json object with two field as  ***(status  "accepted" :  |"rejected") and (reason : )***
       give reason.
        Website Content:
        {website_content}

        Final Answer:
        """

In [ ]:
self.prompt = PromptTemplate(
            input_variables=[self.website_content],
            template="""
        Analyze the following website content and determine if it belongs to a Digital Marketing Agency.

        Look for similar keywords: Services, Web Design, Web Development, SEO Agency, Ads Agency, Digital, Marketing , Agency, Website Creation,
        brands,web,design,Boosts creativity,Reduces costs.

        ***The website content may not mention all the keywords. Use logic and find keywords similar to the above.****

        **if the above keywords or similar keywords are found then it can be a digital marketing agency**
       
       format the output as a json object with two field as  ***(status  "accepted" :  |"rejected") and (reason : )***
       give reason.
       ***Ensure proper json format with opening and closing brackets***
        ***Dont give explanation. Just display only the json object.***

        
      
        Website Content:
        {website_content}

        Final Answer:
        """
        )

In [ ]:
self.prompt = PromptTemplate(
            input_variables=[self.website_content],
            template="""
            Analyze the following website content and classify it.

            Content: {website_content}

            Look for similar keywords: Services, Web Design, Web Development, SEO Agency, Ads Agency, Digital, Marketing , Agency, Website Creation,
            brands,web,design,Boosts creativity,Reduces costs.

            ***The website content may not mention all the keywords. Use logic and find keywords similar to the above.****

            **if the above keywords or similar keywords are found then it can be a digital marketing agency**

            ***Provide the answer in the following JSON format:***
            ```json
            {{
            "Digital Marketing Agency": "Yes" or "No"
            }}
        """
        )

In [ ]:
'''Extract the json from the text given and display only the json part
        text = {text}
        
        output format should be like:
        {{
         "Digital Marketing Agency": "yes/No"
        }}
        ***Extract same json information from the text given. No explanations or extra information and no decorations***
        '''

In [ ]:
##final prompt

self.prompt = PromptTemplate(
            input_variables=[self.website_content],
            template="""You are a classification model. Your task is to analyze the given website content {website_content} and determine if it belongs to a **Digital Marketing Agency**.  

            ### **Classification Logic**
            - Look for keywords related to digital marketing such as:
            * Services, Web Design, Web Development, SEO Agency, Ads Agency, Digital, Marketing, Agency, Website Creation, brands, web, design, Boosts creativity, Reduces costs.
            - You may also detect similar keywords that imply digital marketing services.

            ### **Strict JSON Output Format**  
            ONLY return a valid JSON object in the format below. **No explanations, no additional text—just the JSON.**

            
            {{
            "Digital Marketing Agency": "Yes" or "No",
            "Confidence Scores": [score1, score2]  // First score → Digital Marketing, Second score → Not Digital Marketing
            }}

        Example Outputs

        For a Digital Marketing Agency, return:
        {{
        "Digital Marketing Agency": "Yes",
        "Confidence Scores": [99.00, 1.00]
        
        }}

        For Not a Digital Marketing Agency, return:
        {{
        "Digital Marketing Agency": "No",
        "Confidence Scores": [1.00, 99.00]
        }}

       """ )


In [1]:
import asyncio
from crawl4ai import *


async def main():

    url="https://huggingface.co/"
    # email="masterofkings2023@gmail.com" 
    # curr_dt = datetime.now()
    async with AsyncWebCrawler() as crawler:
        result = await crawler.arun(
            url=url,
        )
       
        f=open('markdown.txt','w',encoding="utf-8")
        f.write(result.markdown)
        f.close()

if __name__ == "__main__":
    asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop